In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
import os

In [3]:
import pickle

In [4]:
import pylangacq as pla

In [5]:
import random

In [59]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F


In [122]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [60]:
import nltk

Load Targets

In [6]:
# y_cookie = torch.load('cookie_data_targets.pt')

In [6]:
os.getcwd()

'/Users/renee/Documents/CT_Fa18/Spec/Trace/FinalProj'

In [9]:
os.chdir('../')

In [22]:
y_mem = torch.load('mem_mmse_dict.pt')

In [23]:
len(y_mem)

46

In [24]:
y_tech = torch.load('tech_mmse_dict.pt')

In [25]:
len(y_tech)

50

In [26]:
y_tech.keys()

dict_keys([6, 29, 51, 56, 78, 89, 100, 107, 118, 154, 184, 202, 217, 226, 234, 242, 266, 277, 280, 286, 297, 303, 324, 337, 347, 352, 359, 375, 387, 398, 411, 423, 433, 438, 445, 448, 460, 464, 470, 474, 477, 496, 499, 502, 506, 509, 519, 522, 537, 540])

In [27]:
apc_datasheet = pd.read_excel('CogData_FU_82818.xlsx',header=None)

In [28]:
apc_datasheet.iloc[0]

0                                             Record ID
1                                            Event Name
2     Write a short sketch about a memory from your ...
3     How does technology and social media impact th...
4                   Date of Neurobehavioral Status Exam
                            ...                        
92                         Cog State: One Back Accuracy
93                          Ravens Progressive Matrices
94                                Logical Memory Part B
95                         East Boston Immediate Recall
96                                            Complete?
Name: 0, Length: 97, dtype: object

In [31]:
apc_datasheet.iloc[list(y_mem.keys()),2]

6      When I was a little child I had the same exper...
29     When I was very young, living in the Astoria n...
51     I went to Paris when I was 16.  I was staying ...
56     My husband and I had always wanted to go to Pa...
78     I was born in Hungary. A significant memory is...
89     When I was 16 years old I traveled to Europe b...
100    I have great memories of growing up in huge ho...
107    When I was 8 or 9 years old, I was playing on ...
118    One of my fondest childhood memories is how mu...
154    One of my favorite memories from my childhood ...
184    My favorite pastime as a small child was re-en...
202    I remember going to the park with my dad to pl...
217    At the age of 14, I was the setter on a volley...
226    I have been able to recall - in detail -- memo...
242    I have a very clear memeory of last time I saw...
266    My parents lived in Ft. Myers, Florida, when I...
277    i visited Washington DC with my brother, my gr...
286    I remember finally going

In [33]:
apc_datasheet.iloc[list(y_tech.keys()),3]

6      I stay in touch with family and friends throug...
29     I'm a journalist who has covered the rise of t...
51     My husband and I are on our computers all day ...
56     I'm on the computer all the time, writing or r...
78     My use of social media is minimal. I only use ...
89     I check that on what my children are doing and...
100    Technology has helped me connect to family and...
107    I use a laptop and phone constantly for work a...
118    I do not use any social media sites at this ti...
154    Social media is great to stay in touch with wh...
184    Social Media is a bit of a challenge as I am a...
202    I use email daily for work and it usually cont...
217    Technology is woven into the fabric of our liv...
226    We dont engage on social media. Email exchange...
234    I'm writing correspondence via computer most o...
242    If by Technology we refer  to communication te...
266    Social media not much, since I don'tuse them, ...
277    I work in a business tha

Load Samples

---------------------------------------------------------------

Load Vocab Embeddings

In [35]:
glove_path = './FinalProj/PretrainedWordEmb/'

In [37]:
os.getcwd()

'/Users/renee/Documents/CT_Fa18/Spec/Trace'

In [39]:
vocab_emb = pickle.load(open(f'{glove_path}/trace.vocab.glove.42B.300_words.pkl', 'rb'))

In [40]:
len(vocab_emb)

2125

In [43]:
trace_addb_posdict = torch.load('apc_pos_dict.pt')

In [44]:
trace_addb_posdict

{'CD': 'DET:NUM',
 'FW': 'FW',
 'EX': 'PRO:EXIST',
 'WP': 'PRO:INT',
 'NNS': 'N',
 'WRB': 'ADV',
 '(': '(',
 'RB': 'ADV',
 'PDT': 'DET:DEM',
 'VBG': 'PART',
 'CC': 'CONJ',
 "''": "''",
 ':': ':',
 'VBP': 'V',
 'IN': 'PREP',
 'TO': 'PREP',
 'MD': 'MOD',
 'PRP$': 'PRO:POSS',
 ')': ')',
 'PRP': 'PRO:PER',
 'JJ': 'ADJ',
 '.': '.',
 'JJR': 'ADJ',
 'VBD': 'V',
 'RBR': 'ADV',
 'RP': 'PART',
 ',': ',',
 'VB': 'V',
 'VBN': 'PART',
 'NN': 'N',
 'NNPS': 'N:PROP',
 'WDT': 'DET:POSS',
 'POS': 'DET:POSS',
 '$': '$',
 'NNP': 'N:PROP',
 'JJS': 'ADJ',
 'VBZ': 'V',
 'RBS': 'ADV',
 'DT': 'DET:ART'}

In [49]:
pos_dict = torch.load('./FinalProj/pos_dict.pt')

In [50]:
pos_dict

{'': 0,
 'CM': 1,
 'COMP': 2,
 'PRO:EXIST': 3,
 'IN#ADJ': 4,
 'PRE#PART': 5,
 '.': 6,
 'PRE#V': 7,
 'OVER#PART': 8,
 'UN#ADV': 9,
 'PRO:PER': 10,
 'ADV:TEM': 11,
 '+/.': 12,
 'V': 13,
 'PRO:INT': 14,
 'N': 15,
 'PRO:POSS': 16,
 'N:ADJ': 17,
 'DET:DEM': 18,
 'END': 19,
 'PRO:INDEF': 20,
 '+"/.': 21,
 'DET:POSS': 22,
 'OVER#V': 23,
 'DET:ART': 24,
 'INF': 25,
 'OVER#N:GERUND': 26,
 'META': 27,
 'ON': 28,
 'UP#V': 29,
 'MID#N': 30,
 'NEG': 31,
 'MOD': 32,
 '+...': 33,
 'PRO:SUB': 34,
 'UN#PART': 35,
 'PREP': 36,
 'DET:NUM': 37,
 'N:GERUND': 38,
 '+//?': 39,
 'UN#ADJ': 40,
 'COP': 41,
 'N:PT': 42,
 'AUX': 43,
 'ADV': 44,
 'OUT#PART': 45,
 '!': 46,
 'PRO:OBJ': 47,
 'ADJ': 48,
 'N:PROP': 49,
 'MINI#N': 50,
 '+/?': 51,
 '?': 52,
 'POST': 53,
 'N:LET': 54,
 'UN#N': 55,
 'UP#PART': 56,
 'COORD': 57,
 'QN': 58,
 'GRAND#N': 59,
 'CONJ': 60,
 '+..?': 61,
 'PART': 62,
 'BEG': 63,
 'PRO:REL': 64,
 'PRO:REFL': 65,
 'NEO': 66,
 'CO': 67,
 '+".': 68,
 'PRO:DEM': 69,
 '+//.': 70}

In [21]:
num_samples

792

-------------------------------------------

Group minibatches of similar size 

In [163]:
# file index: sequence length
ix_size_dict = {}
minibatch_ix = train_ix
    
#index, filename
cookie_files = [(i,X_cookie_filenames[i]) for i in minibatch_ix if i in X_cookie_filenames.keys()]
sent_files = [(i,X_sent_filenames[i] )for i in minibatch_ix if i in X_sent_filenames.keys()]


for corpus,data,targetdict in [(cookie_files,X_cookie_reader,y_cookie),(sent_files,X_sentence_reader,y_sentence)]: 
    for file_ix,file in corpus:
#             print(file_ix, file)
#             print('Words',[tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])
#             print('Words_len',len([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance]))
        embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])]
        ix_size_dict[file_ix] = len(embedding)
#         target = targetdict[file_ix]

In [171]:
# sequence length: [file indices]
size_ix_dict = {}
for k,v in ix_size_dict.items():
    if v not in size_ix_dict.keys():
        size_ix_dict[v] = [k]
    else:
        size_ix_dict[v].append(k)

In [291]:
# set of sequence-lengths
len_sents = sorted(list(size_ix_dict.keys()),reverse=True)

In [220]:
batches = []
while len(len_sents) > 0:
    minibsize = 0
    batchset = []
    while minibsize < 25:
        batchset.extend(size_ix_dict[len_sents[0]])
        minibsize = len(batchset)
        len_sents.pop(0)
    batches.append(batchset)

IndexError: list index out of range

In [223]:
# append final batchset
batches.append(batchset)

In [232]:
# confirm all train files accounted for 
sum([1 for blist in batches for bitem in blist])

634

In [233]:
len(train_ix)

634

-------------------------------------------

In [27]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [53]:
apc_datasheet[2]

0      Write a short sketch about a memory from your ...
1                                                    NaN
2                                                    NaN
3                                                    NaN
4                                                    NaN
                             ...                        
572                                                  NaN
573                                                  NaN
574                                                  NaN
575                                                  NaN
576                                                  NaN
Name: 2, Length: 577, dtype: object

In [177]:
#pad with zeros -- maybe
# end with eos tensor, tag

def get_embs(batchsize=1, mem_ix=[],tech_ix=[],pad=True):
    
    def list_tokenize(document):
#     return re.sub('['+string.punctuation.replace('\'','')+']',' ',response).split()
        sentences = nltk.sent_tokenize(document)

        sentences = [nltk.word_tokenize(sent) for sent in sentences]
        return [token for sentence in sentences for token in sentence]
    
    def get_pos(document):
        #sentence segmentation
        sentences = nltk.sent_tokenize(document) 
        #tokenization
        sentences = [nltk.word_tokenize(sent) for sent in sentences]
        #pos tagging
        sentences = [nltk.pos_tag(sent) for sent in sentences] 

        pos_list = [p for sent in sentences for (t,p) in sent]
        return pos_list

    def num_vectorize(targets):
        vectors = []
        for i in targets:
            v = torch.zeros([30])
            # up until MMSE index...greater than index i 
            v[:i] = 1
            vectors.append(v)
        return torch.stack(vectors).double()
    
    def pad_minibatch(minib):
    
        batchsize = len(minib)

        seq_lens = [len(emb) for emb in minib]
        max_len = max(seq_lens)

        # input of shape (seq_len, batch, input_size): tensor containing the features of the input sequence
        # https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
    #     seq_tensor = torch.zeros((batchsize, max_len, EMBEDDING_SIZE)).double()
        seq_tensor = torch.zeros((max_len, batchsize, EMBEDDING_SIZE)).double()


        for i, (seq,length) in enumerate( zip(minib,seq_lens) ):
            for wi,word in enumerate(seq):
    #             seq_tensor[i,wi] = word
                seq_tensor[wi,i] = word
        # mod to sort at the start?
        seq_tensor = nn.utils.rnn.pack_padded_sequence(seq_tensor,lengths=seq_lens,enforce_sorted=False)
        return seq_tensor
    
#     minibatch_ix = random.sample(range(num_samples),batchsize) if ix is None else ix 
    
    #index, filename
#     cookie_files = [(i,X_cookie_filenames[i]) for i in minibatch_ix if i in X_cookie_filenames.keys()]
#     sent_files = [(i,X_sent_filenames[i] )for i in minibatch_ix if i in X_sent_filenames.keys()]
    
    embeddings = []
    targets = []


    for ix_list,data,targetdict in [(mem_ix,apc_datasheet[2],y_mem),(tech_ix,apc_datasheet[3],y_tech)]:
        
        for ix in ix_list:
#             print('ix',ix,'data',data.iloc[ix],'targetdict',targetdict)
#             return data.iloc[ix]
            embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),trace_addb_posdict[pos]) for (token,pos) in zip(list_tokenize(data.iloc[ix]), get_pos(data.iloc[ix]))]
            
#             print('embedding',embedding)
            target = targetdict[ix]
  # RM POS test  
            emb = []
            for tkn in embedding:
                if tkn[2] in pos_dict:
                    tkn[1][pos_dict[tkn[2]]] = 1
                    # parts of speech ['FW', '(', "''", ':', ')', ',', '$'] not in addb dataset -- rm corresponding tokens  
                    # see nltk.help.upenn_tagset() for parts of speech
                    # leaving in these tokens does not change accuracy but does increase average distance by 1
                    # removing all parts of speech reduces the accuracy and increases the average distance, indicating model relies on pos tag
                    emb.append(torch.cat((tkn[0],tkn[1])))
                    
#                 else:
                    
#                     print(tkn[2] +' not in pos_dict')

    #                 embeddings.append(torch.cat((tkn[0],tkn[1])))
            embeddings.append(emb)
#             embeddings.append([torch.cat((tkn[0],tkn[1])) for tkn in embedding])
            targets.append(target)
#             print(embeddings)
#             print(targets)
#     for corpus,data,targetdict in [(cookie_files,X_cookie_reader,y_cookie),(sent_files,X_sentence_reader,y_sentence)]: 
#         for file_ix,file in corpus:
# #             print(file_ix, file)
# #             print('Words',[tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])
# #             print('Words_len',len([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance]))
#             embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])]
#             target = targetdict[file_ix]
            
#             for tkn in embedding:

#                 tkn[1][tkn[2]] = 1

# #                 embeddings.append(torch.cat((tkn[0],tkn[1])))
            
#             embeddings.append([torch.cat((tkn[0],tkn[1])) for tkn in embedding])


# #             embeddings.append(embedding)
#             targets.append(target)
#     print(targets)       
#     return embeddings, torch.tensor(targets), minibatch_ix
#     return embeddings, torch.tensor(targets)
    if pad: embeddings = pad_minibatch(embeddings)
# return target vals for accuracy 
    return embeddings, num_vectorize(targets), torch.tensor(targets)

In [73]:
list(y_tech.keys())[:5]

[6, 29, 51, 56, 78]

In [81]:
def list_tokenize(document):
#     return re.sub('['+string.punctuation.replace('\'','')+']',' ',response).split()
    sentences = nltk.sent_tokenize(document)

    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    return [token for sentence in sentences for token in sentence]

In [82]:
list_tokenize(apc_datasheet[2].iloc[6])

['When',
 'I',
 'was',
 'a',
 'little',
 'child',
 'I',
 'had',
 'the',
 'same',
 'experience',
 'over',
 'and',
 'over',
 '.',
 'It',
 'always',
 'occurred',
 'between',
 'being',
 'awake',
 'and',
 'asleep',
 '.',
 'The',
 'experience',
 'had',
 'no',
 'words',
 '...',
 'it',
 'was',
 'all',
 'sensation',
 '.',
 'At',
 'a',
 'point',
 'between',
 'my',
 'eyes',
 'I',
 'got',
 'a',
 'feeling',
 'of',
 'all',
 'space',
 'expanding',
 'and',
 'expanding',
 'until',
 'it',
 'became',
 'so',
 'vast',
 'I',
 'was',
 'terrified',
 '.',
 'Then',
 'all',
 'space',
 'would',
 'contract',
 'until',
 'it',
 'became',
 'so',
 'tiny',
 'I',
 'would',
 'be',
 'terrified',
 '.',
 'I',
 'was',
 'both',
 'afraid',
 'of',
 'having',
 'this',
 'experience',
 'and',
 'welcomed',
 'it',
 '.']

In [178]:
e,_,t = get_embs(tech_ix=list(y_tech.keys())[:1],mem_ix=list(y_mem.keys())[:1])

In [181]:
e[0].size()

torch.Size([97, 371])

In [182]:
_

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.]], dtype=torch.float64)

In [183]:
t

tensor([28, 28])

In [99]:
e[0]

tensor([[ 0.2964, -0.3809, -0.3653,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0435, -0.1848, -0.1461,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1204,  0.0567, -0.4998,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.2868,  0.1821, -0.0600,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2773, -0.1023,  0.1760,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0068, -0.0024,  0.0248,  ...,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64)

In [107]:
# https://towardsdatascience.com/simple-trick-to-train-an-ordinal-regression-with-any-classifier-6911183d2a3c
# transform proba dist prediction to MMSE target
def out_to_score_proba(yhat_vector):
    # mod to 31 for ix 0-30
    proba_vector = torch.zeros([31])
    # no zero-score
    proba_vector[0] = 0. 
# P(MMSE=i) = P(MMSE>i-1) - P(MMSE>i)
    for i in range(1,len(yhat_vector)):
        proba_vector[i] = yhat_vector[i-1] - yhat_vector[i]
        
    proba_vector[-1] = yhat_vector[-1] 
    
#     print(proba_vector)
    return torch.argmax(proba_vector)


In [108]:
# not on y but on targets as mmse scores
def accuracy_distansum(yhat_tensor,y_tensor):
    return (torch.abs(y - yhat).sum())

In [109]:
# raw accurate prediction count
def accuracy(yhat_tensor,y_tensor):
#     return torch.stack([ya==yb for (ya,yb) in zip(yhat_tensor,y_tensor)]).sum()
    return ((yhat_tensor == y_tensor).sum())

------------------------------

MODEL 

2-layer rnn: gru units

In [33]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [110]:
HIDDEN_SIZE = 128
# EMBEDDING_SIZE = 371
EMBEDDING_SIZE = len(vocab_emb['I']) + len(addb_pos_dict)

In [268]:
# max([len(emb) for emb in e])

546

In [111]:
# change if aadd in EOS, pos to 547
MAX_LENGTH = 546

In [112]:
VOCAB_SIZE = len(vocab_emb)

In [113]:
BATCH_SIZE = 32

In [114]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size=MAX_LENGTH, emb_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE):
        super(EncoderRNN, self).__init__()
        
        self.emb_size = emb_size
        self.hidden_size = hidden_size

        self.gru = nn.GRU(input_size=self.emb_size,hidden_size=self.hidden_size,num_layers=2).double()
#         # each output node Oi of our neural network
# uses a standard sigmoid function 1
# 1+e−zi
# , without including
# the outputs from other nodes, as shown in Figure 1. Output
# node Oi
# is used to estimate the probability oi
# that a data
# point belongs to category i independently, without subjecting
# to normalization as traditional neural networks do. Thus,
# for a data point x of category k, the target vector is
# (1, , 1, .., 1, 0, 0, 0), in which the first k elements is 1 and
# others 0
# http://orca.st.usm.edu/~zwang/files/rank.pdf A Neural Network Approach to Ordinal Regression
    
        self.fc = nn.Linear(self.hidden_size*2,30).double()
        
        self.activation = nn.Sigmoid()

    def forward(self, input, hidden):

        embedded = input.double()

        gru_out, _ = self.gru(embedded,hidden)
        
        hid_out = torch.cat((_[-2,:,:], _[-1,:,:]), dim = 1)

        out = self.fc(hid_out)
#         print(out.size())

        out - self.activation(out)

        return out, _
#         return torch.nn.utils.rnn.pad_packed_sequence(out), _

    def initHidden(self,batch_size=BATCH_SIZE):
        # h_0 of shape (num_layers * num_directions, batch, hidden_size): tensor containing the initial hidden state for each element in the batch. If the LSTM is bidirectional, num_directions should be 2
        return torch.zeros(2, batch_size, self.hidden_size, device=device).double()
        


In [108]:
# The
# cost function for a data point x can be relative entropy
# or square error between the target vector and the output
# vector
# loss = nn.BCELoss()
loss_func = nn.MSELoss()

In [115]:
encoder = EncoderRNN()

In [116]:
optimizer = torch.optim.Adam(encoder.parameters(), lr=1e-2)

In [321]:
encoder.train()
# _hidden = encoder.initHidden()
# Group batches/paddings by relative same-size

accuracy_floor = .6
distance_floor = 3.6

for epoch in range(25,27):
    
    # modify for ordering -- benefit here with shuffling at ea epoch
#     random.shuffle(train_ix)
    random.shuffle(batches)
    
    print('Epoch',epoch)
    print('===========')
    step = 0
    
    distance = 0
    accurate = 0
    
    # in future, mix up epochs 
    for i in range(len(batches)):
        batch = batches[i]
#     for i in range(19):
        
        step += 1
        optimizer.zero_grad()
        
#         x,y,targets = get_minibatch(batchsize=BATCH_SIZE)
        x,y,targets = get_minibatch(batchsize=len(batch))
#         x = pad_minibatch(x)
        
#         y = torch.stack(y).double()
        
#         _hidden = encoder.initHidden(batch_size=BATCH_SIZE)
        _hidden = encoder.initHidden(batch_size=len(batch))

        yhat, _hidden = encoder(x, _hidden)

        loss = loss_func(yhat,y)

        loss.backward()
#         loss.backward(retain_graph=True)
        
        optimizer.step()

       
        print('step',step)
        print('loss',loss)

        yhat_tensor = torch.tensor([out_to_score_proba(hat) for hat in yhat])
        
        _stepdistance = accuracy_distansum(yhat_tensor=yhat_tensor,y_tensor=targets)
        distance += _stepdistance
        
        _stepaccurate = accuracy(yhat_tensor=yhat_tensor,y_tensor=targets)
        accurate += _stepaccurate
        
        
        print('step_accuracy_distance',_stepdistance)
        print('step_accurate',_stepaccurate)
    
    print('Accurate_total',accurate)
    print('Distance_total',distance)
    
    epoch_accuracy = accurate.item()/(len(train_ix))
    # may be a bug in distance..
    epoch_distance_avg = distance.item()/(len(train_ix))
    
    print('Epoch_accuracy',epoch_accuracy)
    print('Epoch_distance_avg',epoch_distance_avg)
    
    if epoch_accuracy > accuracy_floor or epoch_distance_avg < distance_floor:
        
        print('Saving..')
        torch.save(encoder.state_dict(),'encoder_accuracy_{:.3f}_avgdistance_{:3f}.pt'.format(epoch_accuracy, epoch_distance_avg))
        
        if epoch_accuracy > accuracy_floor: accuracy_floor = epoch_accuracy
        if epoch_distance_avg > distance_floor: distance_floor = epoch_distance_avg
        
    
        
    
    
    

Epoch 25
torch.Size([26, 30])
step 1
loss tensor(0.0036, dtype=torch.float64, grad_fn=<MseLossBackward>)
step_accuracy_distance tensor(31.8367, dtype=torch.float64, grad_fn=<SumBackward0>)
step_accurate tensor(26)
torch.Size([13, 30])
step 2
loss tensor(0.0037, dtype=torch.float64, grad_fn=<MseLossBackward>)
step_accuracy_distance tensor(15.9313, dtype=torch.float64, grad_fn=<SumBackward0>)
step_accurate tensor(13)
torch.Size([26, 30])
step 3
loss tensor(0.0036, dtype=torch.float64, grad_fn=<MseLossBackward>)
step_accuracy_distance tensor(32.3432, dtype=torch.float64, grad_fn=<SumBackward0>)
step_accurate tensor(26)
torch.Size([32, 30])
step 4
loss tensor(0.0035, dtype=torch.float64, grad_fn=<MseLossBackward>)
step_accuracy_distance tensor(37.2153, dtype=torch.float64, grad_fn=<SumBackward0>)
step_accurate tensor(32)
torch.Size([25, 30])
step 5
loss tensor(0.0031, dtype=torch.float64, grad_fn=<MseLossBackward>)
step_accuracy_distance tensor(27.8402, dtype=torch.float64, grad_fn=<SumBac

In [168]:
encoder.load_state_dict(torch.load('./FinalProj/encoder_0.99accuracy_25epch.pt'))

<All keys matched successfully>

In [169]:
encoder.eval()

EncoderRNN(
  (gru): GRU(371, 128, num_layers=2)
  (fc): Linear(in_features=256, out_features=30, bias=True)
  (activation): Sigmoid()
)

In [184]:
x_trace,y_trace,targets_trace = get_embs(mem_ix=list(y_mem.keys()),tech_ix=list(y_tech.keys()))

In [187]:
len(y_mem) + len(y_tech)

96

In [188]:
_hidden = encoder.initHidden(batch_size=len(y_mem)+len(y_tech))

In [189]:
yhat, _hidden = encoder(x_trace, _hidden)
# Rm non-pos
estimates = torch.tensor([out_to_score_proba(hat) for hat in yhat])

In [190]:
accuracy(targets_trace,estimates)

tensor(9)

In [191]:
(torch.abs(targets_trace - estimates).sum())/len(estimates)

tensor(4)

In [192]:
estimates

tensor([29, 29, 28, 28, 23, 28, 29, 23, 16, 29, 28, 16,  3, 28, 28, 29, 30, 29,
        19, 28, 28, 29, 28, 29, 28, 23, 16, 24, 20, 29, 29, 13, 29, 28, 29, 28,
        23, 30, 20, 28, 26, 28, 28, 28, 23, 28, 23, 28,  5, 19, 19, 29, 19, 20,
        28, 19, 28, 28, 28, 20, 29, 30, 28, 28, 23, 19, 19, 28, 18, 17, 28, 28,
        25, 29, 28, 20, 28, 20, 19, 16, 19, 28, 23, 28, 28, 20, 28, 19, 28, 28,
        28, 23, 21, 29, 29, 28])

In [193]:
targets_trace

tensor([28, 30, 28, 30, 30, 30, 30, 29, 29, 29, 30, 30, 30, 29, 30, 30, 30, 30,
        28, 29, 30, 28, 30, 29, 30, 26, 30, 29, 29, 30, 30, 29, 30, 28, 30, 30,
        30, 30, 30, 30, 30, 30, 29, 30, 30, 29, 28, 30, 28, 30, 30, 30, 30, 29,
        29, 29, 30, 30, 30, 29, 29, 30, 30, 30, 30, 30, 28, 29, 30, 28, 28, 30,
        29, 30, 26, 30, 29, 29, 30, 30, 29, 30, 28, 30, 30, 30, 30, 30, 30, 30,
        30, 29, 30, 30, 30, 29])

In [161]:
6/96

0.0625

In [130]:
9/96

0.09375

In [162]:
# Without pos tags - ix 13 'I have been able to recall - in detail -- memories from my first year.  This was confirmed by my Mother.' jumps from 3 to 19 
# ...but pos is imp...tweak pos output?
estimates

tensor([29, 29, 20, 28, 23, 29, 29, 23, 16, 29, 28, 16, 19, 28, 28, 29, 28, 19,
        19, 25, 28, 29, 17, 29, 28, 23, 16, 24, 20, 29, 17, 29, 29, 28, 29, 28,
        23, 30, 29, 28, 26, 28, 28, 19, 14, 28, 23, 16,  5, 25, 13, 29, 19, 19,
        25, 19, 19,  3, 28, 20, 25, 30, 19, 29, 26, 19, 14, 28, 18, 17, 20, 28,
        16, 29, 28, 15, 28, 20,  5, 16, 19, 20, 23, 24, 28, 20, 13, 19, 16, 28,
        28, 23, 28, 19, 29, 28])

In [163]:
(torch.abs(targets_trace - estimates).sum())/len(estimates)

tensor(6)

In [126]:
estimates

tensor([29, 30, 28, 28, 23, 29, 29, 23, 16, 29, 28, 16,  3, 28, 28, 29, 28, 19,
        19, 28, 28, 29, 28, 29, 28, 23, 16, 24, 20, 29, 28, 13, 29, 28, 29, 28,
        23, 30, 20, 28, 26, 28, 28, 28, 23, 28, 23, 16,  5, 19, 13, 29, 19, 11,
        28, 19, 19,  3, 28, 28, 29, 30, 19, 28, 23, 19, 14, 28, 18, 17, 28, 28,
        16, 29, 28, 15, 28, 20,  3, 16, 19, 28, 23, 24, 28, 20, 28, 19, 28, 28,
        28, 23, 21, 29, 29, 28])

In [196]:
estimates[48]

tensor(5)

In [200]:
apc_datasheet.iloc[list(y_tech.keys())[3],3]

"I'm on the computer all the time, writing or responding to social media. I've made friends. I keep in touch with friends. The bad thing about social media, though, is that it's an easy distraction. If the writing is going badly, it's all too easy to pop over to Facebook and pick a fight about politics."

In [132]:
targets_trace

tensor([28, 30, 28, 30, 30, 30, 30, 29, 29, 29, 30, 30, 30, 29, 30, 30, 30, 30,
        28, 29, 30, 28, 30, 29, 30, 26, 30, 29, 29, 30, 30, 29, 30, 28, 30, 30,
        30, 30, 30, 30, 30, 30, 29, 30, 30, 29, 28, 30, 28, 30, 30, 30, 30, 29,
        29, 29, 30, 30, 30, 29, 29, 30, 30, 30, 30, 30, 28, 29, 30, 28, 28, 30,
        29, 30, 26, 30, 29, 29, 30, 30, 29, 30, 28, 30, 30, 30, 30, 30, 30, 30,
        30, 29, 30, 30, 30, 29])

In [135]:
(torch.abs(targets_trace - estimates).sum())/len(estimates)

tensor(5)

In [128]:
len(estimates)

96

In [152]:
apc_datasheet.iloc[list(list(y_mem.keys())+list(y_tech.keys()))[13],2]

'I have been able to recall - in detail -- memories from my first year.  This was confirmed by my Mother.'

In [150]:
apc_datasheet.iloc[list(list(y_mem.keys())+list(y_tech.keys()))[78],3]

"I don't use social media, as I hear that it consumes a lot of time.  I only use email.  I don't have a Facebook account nor any other social media account.  I use my iphone regularly for email, calendar and notes fields, as well as the internet.    I don't feel the need to look into people's lives via social media, as I feel it could be depressing.  I'd rather email, speak or visit with friends or family."

In [146]:
estimates[78]

tensor(3)

In [201]:
accuracy(targets_trace[:46],estimates[:46])

tensor(6)

In [140]:
6/46

0.13043478260869565

In [143]:
(torch.abs(targets_trace[:46] - estimates[:46]).sum())/46

tensor(4)

In [202]:
accuracy(targets_trace[46:],estimates[46:])

tensor(3)

In [142]:
3/50

0.06

In [144]:
(torch.abs(targets_trace[46:] - estimates[46:]).sum())/50

tensor(7)

In [145]:
1/30

0.03333333333333333

In [127]:
accuracy_distansum(targets_trace,estimates)

NameError: name 'y' is not defined

In [255]:

encoder.eval()

x_valid,y_valid,targets_valid = get_minibatch(ix=valid_ix)

# x_valid = pad_minibatch(x_valid)

# y_valid = torch.stack(y_valid).double()

_hidden = encoder.initHidden(batch_size=len(valid_ix))

yhat_valid, _hidden = encoder(x_valid, _hidden)
estimates = torch.tensor([out_to_score_proba(hat) for hat in yhat_valid])


torch.Size([158, 30])


In [264]:
targets_valid

tensor([30, 18, 29, 28, 26, 27, 23, 30, 19, 17, 20, 17, 25, 30, 23, 29, 27, 30,
        19, 16, 30,  7, 28, 28, 28, 28, 25, 12, 30, 18, 30, 25, 15, 18, 14, 26,
        27, 30, 20, 17, 30, 26, 19, 27, 17, 20, 20, 29, 27, 29, 30, 30, 30, 20,
        22, 29, 19, 30, 30, 30, 30, 22, 17, 27, 27, 30, 30, 23, 30, 30, 19, 25,
        30, 19, 30, 30, 30,  8, 30, 22, 30, 30, 28, 16, 19, 22, 24, 28, 16, 30,
        30, 19, 17, 27, 19, 30, 30, 21, 28, 19, 29, 19, 12, 28, 20, 23, 13, 20,
        23, 27, 23, 19, 20, 19, 19, 19, 27, 16, 10, 20, 17, 24, 24, 23, 16, 20,
        18, 17, 19, 28, 29, 20, 27, 20, 23, 16, 20, 23, 27, 22, 19, 20, 20, 19,
        19, 28, 28, 28, 26, 19, 24, 22, 20, 18, 23, 27, 25, 25])

In [312]:
estimates

tensor([30, 18, 29, 30, 26, 27, 23, 30, 19, 17, 20, 17, 25, 30, 23, 29, 27, 30,
        19, 16, 30,  7, 28, 28, 28, 28, 25, 12, 30, 18, 30, 25, 15, 18, 14, 26,
        27, 30, 20, 17, 30, 26, 19, 27, 17, 20, 20, 29, 27, 29, 30, 30, 30, 20,
        22, 29, 19, 30, 30, 30, 30, 22, 17, 27, 27, 30, 30, 23, 30, 30, 19, 25,
        30, 19, 30, 30, 30,  8, 30, 22, 30, 30, 28, 16, 19, 22, 24, 28, 16, 30,
        30, 19, 17, 27, 19, 30, 30, 21, 28, 19, 29, 19, 12, 28, 20, 23, 13, 20,
        23, 27, 23, 19, 20, 19, 19, 19, 27, 16, 10, 20, 17, 24, 24, 23, 16, 20,
        18, 17, 19, 28, 29, 20, 27, 20, 23, 16, 20, 23, 27, 22, 19, 20, 20, 19,
        19, 28, 28, 28, 26, 19, 24, 22, 20, 18, 23, 27, 25, 25])

In [303]:
# Export for heatmap visualization
# pd.Series(targets_valid).to_csv('targets_valid.csv')

In [304]:
# pd.Series(estimates).to_csv('estimates.csv')

In [256]:
# what is WRONG with this metric
accuracy_distansum(targets_valid,estimates)

tensor(14.6159, dtype=torch.float64, grad_fn=<SumBackward0>)

In [299]:
14.6159/158

0.09250569620253164

In [257]:
accuracy(targets_valid,estimates)

tensor(157)

In [258]:
157/158

0.9936708860759493

In [259]:
np.where(targets_valid != estimates)

(array([3]),)

In [305]:
encoder.eval()

x_train,y_train,targets_train = get_minibatch(ix=train_ix)

# x_valid = pad_minibatch(x_valid)

# y_valid = torch.stack(y_valid).double()

_hidden = encoder.initHidden(batch_size=len(train_ix))
yhat_train, _hidden = encoder(x_train, _hidden)
estimates_train = torch.tensor([out_to_score_proba(hat) for hat in yhat_train])

torch.Size([634, 30])


In [306]:
targets_train

tensor([28, 30, 12, 17, 30, 29, 20, 29, 15, 30, 29, 29, 29, 29, 30, 30, 28,  3,
        28, 27, 30, 19, 27, 30, 10, 30, 30, 17, 23, 30, 30, 19, 30, 30, 30, 30,
        30, 28, 30, 19, 29, 30, 30, 29, 30, 11, 27, 21, 19, 29, 30, 28, 30, 12,
        30, 13, 22, 21, 29, 29, 17, 10, 23, 13, 18, 11, 28, 30, 18,  3, 29, 27,
        30, 21, 20, 25, 30, 28, 30, 17, 24, 26, 20, 27, 30, 30, 19, 28, 24, 25,
        20, 24, 30, 29, 16, 30, 30, 30, 30, 30, 19, 28, 15, 30, 30, 26, 30, 29,
        30, 30, 30, 26, 30, 19, 26, 23, 18, 19, 13, 23, 12, 25, 30, 30, 10, 24,
        30, 30, 28, 28, 15, 30, 30, 30, 24, 29, 28, 30, 28, 30, 23, 23, 20, 30,
        29,  8, 20, 30, 19, 20, 25, 29, 20, 30, 30, 28, 30, 27, 29, 28, 22, 28,
        25, 23, 29, 20, 23, 16, 18, 23, 19, 25, 30, 30, 17, 30, 30, 20, 10, 30,
        18, 22, 24, 15, 29, 30, 19, 18, 28, 30, 30, 29, 30, 30, 30, 22, 29, 30,
        29, 30, 26, 29, 15, 16, 30, 30, 13, 24, 30, 29, 23, 28, 13, 30, 28, 11,
        29, 18, 18, 17, 27, 16, 12, 13, 

In [314]:
estimates_train_bugfix = torch.tensor([out_to_score_proba(hat) for hat in yhat_train])

In [315]:
estimates_train_bugfix

tensor([28, 30, 12, 17, 30, 29, 20, 29, 15, 30, 29, 29, 29, 29, 30, 30, 28,  3,
        28, 27, 30, 19, 27, 30, 10, 30, 30, 17, 23, 30, 30, 19, 30, 30, 30, 30,
        30, 28, 30, 19, 29, 30, 30, 29, 30, 11, 27, 21, 19, 29, 30, 28, 30, 12,
        30, 13, 22, 21, 29, 29, 17, 10, 23, 13, 18, 11, 28, 30, 18,  3, 29, 27,
        30, 21, 20, 25, 30, 28, 30, 17, 24, 26, 20, 27, 30, 30, 19, 28, 24, 25,
        20, 24, 30, 29, 16, 30, 30, 30, 30, 30, 19, 28, 15, 30, 30, 26, 30, 29,
        30, 30, 30, 26, 30, 19, 26, 23, 18, 19, 13, 23, 12, 25, 30, 30, 10, 24,
        30, 30, 28, 28, 15, 30, 30, 30, 24, 29, 28, 30, 28, 30, 23, 23, 20, 30,
        29,  8, 20, 30, 19, 20, 25, 29, 20, 30, 30, 28, 30, 27, 29, 28, 22, 28,
        25, 23, 29, 20, 23, 16, 18, 23, 19, 25, 30, 30, 17, 30, 30, 20, 10, 30,
        18, 22, 24, 15, 29, 30, 19, 18, 28, 30, 30, 29, 30, 30, 30, 22, 29, 30,
        29, 30, 26, 29, 15, 16, 30, 30, 13, 24, 30, 29, 23, 28, 13, 30, 28, 11,
        29, 18, 18, 17, 27, 16, 12, 13, 

In [316]:
accuracy(targets_train,estimates_train_bugfix)

tensor(632)

In [317]:
632/len(train_ix)

0.9968454258675079

In [318]:
accuracy_distansum(targets_train,estimates_train_bugfix)

tensor(14.6159, dtype=torch.float64, grad_fn=<SumBackward0>)

In [319]:
14.6159/len(train_ix)

0.02305347003154574